In [1]:
import numpy as np
import pandas as pd

In [2]:
train=pd.read_csv("train.csv")

In [3]:
train.drop('tweet_id', axis=1, inplace=True)
train.drop('name', axis=1, inplace=True)
train.drop('user_timezone',axis=1,inplace=True)
train.drop('tweet_location',axis=1,inplace=True)
train.drop('tweet_coord',axis=1,inplace=True)
train.drop('airline_sentiment_gold',axis=1,inplace=True)
train.drop('negativereason_gold',axis=1,inplace=True)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10980 entries, 0 to 10979
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   airline_sentiment  10980 non-null  object
 1   airline            10980 non-null  object
 2   retweet_count      10980 non-null  int64 
 3   text               10980 non-null  object
 4   tweet_created      10980 non-null  object
dtypes: int64(1), object(4)
memory usage: 429.0+ KB


In [4]:
a=train.tweet_created.copy()

In [5]:
hour_at_which_tweet_created=[int(i.split(" ")[1].split(":")[0]) for i in a]

In [6]:
train.drop('tweet_created',axis=1,inplace=True)

In [7]:
text_raw_data=train.text.copy()

In [8]:
from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
# from nltk import pos_tag
# lemmatizer=WordNetLemmatizer()
stops=set(stopwords.words("english"))
import string
punctuations=list(string.punctuation)
stops.update(punctuations)
stops.update(set([""]))

In [9]:
import spacy
nlp = spacy.load('en', disable=['parser', 'ner'])

In [10]:
Text=[]

In [11]:
for i in text_raw_data:
    refined_text=[]
    for w in i.split(" "):
        if(w.lower() not in stops):
            doc=nlp(w)
            clean_word=[token.lemma_ for token in doc][0]
            if clean_word!='-PRON-':
                refined_text.append(clean_word.lower())
    Text.append(" ".join(refined_text))

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
count_vec=CountVectorizer(max_features=700,max_df=0.95,min_df=0.005,ngram_range=(1,2))
train_text=count_vec.fit_transform(Text)

In [13]:
text_dense_matrix=train_text.todense()
train.drop('text',axis=1,inplace=True)

In [14]:
convert_airlines={"American":0,"Delta":1,"Southwest":2,"US Airways":3,"United":4,"Virgin America":5}
convert_output={"negative":1,"positive":2,"neutral":0}
Airline=[convert_airlines[i] for i in train.airline]
train.drop("airline",axis=1,inplace=True)
y_train=[convert_output[i] for i in train.airline_sentiment]
train.drop("airline_sentiment",axis=1,inplace=True)
train["Airline"]=Airline

In [15]:
train

,retweet_count,Airline
0,0,2
1,0,2
2,0,4
3,0,2
4,0,4
...,...,...
10975,0,0
10976,0,4
10977,0,3
10978,0,3


In [16]:
train=train.to_numpy()

In [17]:
x_final=np.concatenate((train,text_dense_matrix), axis=1)

In [21]:
from sklearn.svm import SVC

In [22]:
scvl=SVC(kernel="rbf")

In [23]:
scvl.fit(x_final,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [24]:
y_predict=scvl.predict(x_final)

In [25]:
from sklearn.metrics import classification_report
print(classification_report(y_train,y_predict))

              precision    recall  f1-score   support

           0       0.69      0.52      0.59      2327
           1       0.83      0.94      0.88      6851
           2       0.80      0.63      0.71      1802

    accuracy                           0.80     10980
   macro avg       0.77      0.70      0.73     10980
weighted avg       0.79      0.80      0.79     10980



In [26]:
from sklearn.naive_bayes import MultinomialNB

In [27]:
NB=MultinomialNB()

In [28]:
NB.fit(x_final,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [29]:
y_predict=NB.predict(x_final)

In [30]:
print(classification_report(y_train,y_predict))

              precision    recall  f1-score   support

           0       0.56      0.53      0.54      2327
           1       0.82      0.86      0.84      6851
           2       0.69      0.65      0.67      1802

    accuracy                           0.75     10980
   macro avg       0.69      0.68      0.68     10980
weighted avg       0.75      0.75      0.75     10980



In [31]:
test=pd.read_csv("test.csv")

In [32]:
test.drop('tweet_id', axis=1, inplace=True)
test.drop('name', axis=1, inplace=True)
test.drop('user_timezone',axis=1,inplace=True)
test.drop('tweet_location',axis=1,inplace=True)
test.drop('tweet_coord',axis=1,inplace=True)
test.drop('airline_sentiment_gold',axis=1,inplace=True)
test.drop('negativereason_gold',axis=1,inplace=True)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3660 entries, 0 to 3659
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   airline        3660 non-null   object
 1   retweet_count  3660 non-null   int64 
 2   text           3660 non-null   object
 3   tweet_created  3660 non-null   object
dtypes: int64(1), object(3)
memory usage: 114.5+ KB


In [33]:
a=test.tweet_created.copy()

In [34]:
hour_at_which_tweet_created=[int(i.split(" ")[1].split(":")[0]) for i in a]

In [35]:
test.drop('tweet_created',axis=1,inplace=True)

In [36]:
text_raw_data=test.text.copy()

In [37]:
Text=[]
for i in text_raw_data:
    refined_text=[]
    for w in i.split(" "):
        if(w.lower() not in stops):
            doc=nlp(w)
            clean_word=[token.lemma_ for token in doc][0]
            if clean_word!='-PRON-':
                refined_text.append(clean_word.lower())
    Text.append(" ".join(refined_text))

In [38]:
test_text=count_vec.transform(Text)
text_dense_matrix=test_text.todense()
test.drop('text',axis=1,inplace=True)

In [39]:
Airline=[convert_airlines[i] for i in test.airline]
test.drop("airline",axis=1,inplace=True)
test["Airline"]=Airline

In [40]:
test=test.to_numpy()
x_final=np.concatenate((test,text_dense_matrix), axis=1)

In [41]:
x_final.shape

(3660, 396)

In [42]:
y_predict=NB.predict(x_final)

In [43]:
convert_output={1:"negative",2:"positive",0:"neutral"}

In [44]:
y_predict=[convert_output[i] for i in y_predict]

In [45]:
np.savetxt('reaction.csv', y_predict, delimiter=',',fmt='%s')